In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
# tag: remove-cell applied

# Dichotomous Data with Bayesian Model Averaging

Bayesian model averaging is currently only available for dichotomous datasets in BMDS. Here, we will go over how to run a dichotomous analysis with Bayesian model averaging and how to plot your results. Also, we will demonstrate how you can override the default priors for parameter estimation.

## Quickstart

To run Bayesian model averaging:

In [ ]:
import pybmds

# create a dichotomous dataset
dataset = pybmds.DichotomousDataset(
    doses=[0, 25, 75, 125, 200],
    ns=[20, 20, 20, 20, 20],
    incidences=[0, 1, 7, 15, 19],
)

session1 = pybmds.Session(dataset=dataset)
session1.add_default_bayesian_models()
session1.execute()

res = session1.model_average.results
print(f"BMD = {res.bmd:.2f} [{res.bmdl:.2f}, {res.bmdu:.2f}]")

session1.plot(colorize=False)

## Changing the input settings

The default settings for a dichotomous Bayesian model averaged run use a BMR of 10% Extra Risk and a 95% confidence interval. You can change these settings by:

In [ ]:
from pybmds.models import dichotomous

session = pybmds.Session(dataset=dataset)
session.add_default_bayesian_models(
    settings = {
        "bmr": 0.05,
        "bmr_type": pybmds.DichotomousRiskType.AddedRisk,
        "alpha": 0.1
    }
)

This would run the dichotomous models for a BMR of 5% Added Risk at a 90% confidence interval.

## Running one Bayesian model

You can also run one model with the default prior distributions for the parameters, rather than running all of the models and finding the model average. If you wanted to run the Logistic model with the default prior distributions, you coudl write the following snippet. Note the model outputs show the prior distributions for `a ~ Normal(0, 2)` and `b ~ Lognormal(0, 2)` and their constraints:

In [ ]:
model = dichotomous.Logistic(dataset=dataset, settings={"priors": pybmds.PriorClass.bayesian})
result = model.execute()
print(model.text())

The default samples and burn-in are 100 and 20, respectively. You can change the burn-ins and samples by:

In [ ]:
model = dichotomous.Logistic(
    dataset=dataset,
    settings={
        "priors": pybmds.PriorClass.bayesian,
        "samples": 1000,
        "burnin": 500
    }
)

The maximum number of samples allowed is 1000.

## Changing parameter settings

Running a single bayesian model with base configuration is shown below:

In [ ]:
model = dichotomous.Logistic(dataset=dataset, settings={"priors": pybmds.PriorClass.bayesian})

print(model.settings.tbl())
print(model.priors_tbl())

Configuration of the model settings or priors can be configured:

In [ ]:
model = dichotomous.Logistic(
    dataset=dataset,
    settings={
        "priors": pybmds.PriorClass.bayesian,
        "samples": 1000,
        "burnin": 500
    }
)
model.settings.priors.update('a', stdev=1, min_value=-15, max_value=15)
model.settings.priors.update('b', stdev=3)

print(model.settings.tbl())
print(model.priors_tbl())

You can also change the parameter prior types from the default listed in the BMDS User Guide. Parameters can be given a Normal, Log-Normal, or Uniform distribution. These can be changed by:

In [ ]:
model = dichotomous.Weibull(
    dataset=dataset,
    settings={
        "priors": pybmds.PriorClass.bayesian,
        "samples": 1000,
        "burnin": 500
    }
)
model.settings.priors.update('g', type=pybmds.PriorDistribution.Uniform, min_value=0, max_value=1)
print(model.priors_tbl())

Or:

In [ ]:
model.settings.priors.update(
    'g', type=pybmds.PriorDistribution.Lognormal, min_value=0, max_value=1, initial_value=0, stdev=1.5
)
print(model.priors_tbl())

## Model average on a subset of models

You can select a set of models to model average on, rather than using all of the default dichotomous models. For example, to model average on the Logistic, Probit, and Weibull models, you can:

In [ ]:
session1 = pybmds.Session(dataset=dataset)
session1.add_model(pybmds.Models.Weibull, {"priors": pybmds.PriorClass.bayesian})
session1.add_model(pybmds.Models.Logistic, {"priors": pybmds.PriorClass.bayesian})
session1.add_model(pybmds.Models.Probit, {"priors": pybmds.PriorClass.bayesian})
session1.add_model_averaging()

session1.execute()

res = session1.model_average.results
print(f"BMD = {res.bmd:.2f} [{res.bmdl:.2f}, {res.bmdu:.2f}]")

session1.plot(colorize=False)